In [1]:
import pandas as pd
import numpy as np

In [2]:


customers_df = pd.read_csv("Customers.csv")
products_df = pd.read_csv("Products.csv")
transactions_df = pd.read_csv("Transactions.csv")

customers_df = customers_df.dropna(axis=1, how='any')
products_df = products_df.dropna(axis=1, how='any')
transactions_df = transactions_df.dropna(axis=1, how='any')

customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

merged_df = transactions_df.merge(customers_df, on='CustomerID')
merged_df = merged_df.merge(products_df, on='ProductID')

merged_df['Year'] = merged_df['TransactionDate'].dt.year

print("\nAvailable Years for Each Region:")
print(merged_df.groupby('Region')['Year'].unique())


Available Years for Each Region:
Region
Asia             [2024, 2023]
Europe           [2024, 2023]
North America    [2024, 2023]
South America          [2024]
Name: Year, dtype: object


In [3]:


# 1. Revenue Distribution: Regional revenue and growth trends
region_revenue = merged_df.groupby('Region')['TotalValue'].sum().reset_index()
region_revenue['Percentage'] = (region_revenue['TotalValue'] / region_revenue['TotalValue'].sum()) * 100
print("\nRevenue Distribution by Region:")
print(region_revenue)

# Simulating Asia growth acceleration (replace with actual yearly comparison if data allows)
merged_df['Year'] = merged_df['TransactionDate'].dt.year
asia_growth = merged_df[merged_df['Region'] == 'Asia'].groupby('Year')['TotalValue'].sum().pct_change().iloc[-1] * 100
print(f"\nAsia revenue growth rate: {asia_growth:.2f}% annually")

merged_df['Year'] = merged_df['TransactionDate'].dt.year
Europe_growth = merged_df[merged_df['Region'] == 'Europe'].groupby('Year')['TotalValue'].sum().pct_change().iloc[-1] * 100
print(f"\nEurope revenue growth rate: {Europe_growth:.2f}% annually")

merged_df['Year'] = merged_df['TransactionDate'].dt.year
na_growth = merged_df[merged_df['Region'] == 'North America'].groupby('Year')['TotalValue'].sum().pct_change().iloc[-1] * 100
print(f"\nNorth America revenue growth rate: {na_growth:.2f}% annually")

merged_df['Year'] = merged_df['TransactionDate'].dt.year
sa_growth = merged_df[merged_df['Region'] == 'South America'].groupby('Year')['TotalValue'].sum().pct_change().iloc[-1] * 100
print(f"\nSouth America revenue growth rate: {sa_growth:.2f}% annually")

# 2. Loyalty: Revenue from repeat customers
repeat_customers = merged_df.groupby('CustomerID').agg(
    TotalRevenue=('TotalValue', 'sum'),
    TransactionCount=('TransactionID', 'count')
).reset_index()
repeat_customers['RepeatCustomer'] = repeat_customers['TransactionCount'] > 1
repeat_revenue = repeat_customers[repeat_customers['RepeatCustomer']]['TotalRevenue'].sum()
total_revenue = repeat_customers['TotalRevenue'].sum()
top_customers_revenue = repeat_customers.nlargest(10, 'TotalRevenue')['TotalRevenue'].sum()

repeat_revenue_percentage = (repeat_revenue / total_revenue) * 100
top_customers_percentage = (top_customers_revenue / total_revenue) * 100

print(f"\nPercentage of revenue from repeat customers: {repeat_revenue_percentage:.2f}%")
print(f"Top customers contribute {top_customers_percentage:.2f}% of total revenue")

# 3. Category Trends: Best-selling categories and growth rates
category_revenue = merged_df.groupby('Category')['TotalValue'].sum().reset_index()
category_revenue['Percentage'] = (category_revenue['TotalValue'] / category_revenue['TotalValue'].sum()) * 100

# Simulate growth rate (replace with actual yearly comparison if available)
home_decor_growth = merged_df[merged_df['Category'] == 'Home Decor'].groupby('Year')['TotalValue'].sum().pct_change().iloc[-1] * 100

print("\nCategory Revenue Distribution:")
print(category_revenue)
print(f"Home Decor revenue growth rate: {home_decor_growth:.2f}% annually")

# 4. Seasonality: Q4 sales trends
merged_df['Quarter'] = merged_df['TransactionDate'].dt.to_period('Q')
quarterly_revenue = merged_df.groupby('Quarter')['TotalValue'].sum().reset_index()

# Extract Q4 revenue
q4_revenue = quarterly_revenue[quarterly_revenue['Quarter'].astype(str).str.endswith('Q4')]['TotalValue'].sum()
q4_percentage = (q4_revenue / quarterly_revenue['TotalValue'].sum()) * 100
december_revenue = merged_df[merged_df['TransactionDate'].dt.month == 12]['TotalValue'].sum()
december_percentage = (december_revenue / quarterly_revenue['TotalValue'].sum()) * 100

print(f"\nQ4 revenue contribution: {q4_percentage:.2f}% of yearly revenue")
print(f"December revenue contribution: {december_percentage:.2f}% of yearly revenue")

print(merged_df.columns)

# Average Order Value (AOV) by Region
region_aov = merged_df.groupby('Region').agg(
    TotalRevenue=('TotalValue', 'sum'),
    TotalTransactions=('TransactionID', 'count')
).reset_index()
region_aov['AverageOrderValue'] = region_aov['TotalRevenue'] / region_aov['TotalTransactions']

print("\nAverage Order Value (AOV) by Region:")
print(region_aov[['Region', 'AverageOrderValue']])



Revenue Distribution by Region:
          Region  TotalValue  Percentage
0           Asia   152074.97   22.039993
1         Europe   166254.63   24.095029
2  North America   152313.40   22.074548
3  South America   219352.56   31.790431

Asia revenue growth rate: 9392.46% annually

Europe revenue growth rate: 12862.94% annually

North America revenue growth rate: 16511.29% annually

South America revenue growth rate: nan% annually

Percentage of revenue from repeat customers: 99.08%
Top customers contribute 11.02% of total revenue

Category Revenue Distribution:
      Category  TotalValue  Percentage
0        Books   192147.47   27.847639
1     Clothing   166170.66   24.082859
2  Electronics   180783.50   26.200676
3   Home Decor   150893.93   21.868826
Home Decor revenue growth rate: 47867.64% annually

Q4 revenue contribution: 21.46% of yearly revenue
December revenue contribution: 9.10% of yearly revenue
Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       '